## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
import numpy as np
import pandas as pd
from sklearn import gaussian_process
from matplotlib import pyplot as plt 
from enum import Enum
import os.path

In [ ]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: [tuple(
    np.round(np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=2),5)) for z in range(n)]

In [ ]:
get_random_pos(3)

In [ ]:
## Enum classes
class AgeGroup(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    
    @staticmethod
    def all():
        return([g for g in AgeGroup])
    @property
    def range(self): return self.end - self.start
    
class SummaryNorm(Enum):
    l1= lambda x: sum(abs(x))
    l2= lambda x: (sum(x**2))**0.5
    lInf= lambda x: max(x)
    

class ServiceType(Enum):
    School = (1, SummaryNorm.l2)
    SocialSupport = (2, SummaryNorm.l2)
    PoliceStation = (2, SummaryNorm.l2)
    #etc
    def __init__(self, _, aggrNormInput=SummaryNorm.l2):
        self.aggrNorm = aggrNormInput
        # initialise demand factors for each age group
        self.demandFactors = pd.Series({a: np.random.uniform() for a in AgeGroup.all()}) #TODO: import this from input
    
    def aggregate_units(self, unitValues, axis=1):
        # assumes positions are stacked in rows
        return np.apply_along_axis(self.aggrNorm, axis, unitValues)
    
    @staticmethod
    def all():
        return([g for g in ServiceType])
    
#demandFactors = pd.DataFrame(np.ones([len(AgeGroup.all()), len(ServiceType.all())]), 
#                             index=AgeGroup.all(), columns=ServiceType.all())

In [ ]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=(45.4641, 9.1919), users=AgeGroup.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        assert isinstance(service, ServiceType), 'Service must belong to the Enum'
        self.name = name
        self.service = service
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        self.times = times
        
        # how the service availablity area varies for different age groups
        kPropagationTest = 3
        self.propagation ={g: (0.04 + .001*np.round(np.random.normal(),2))*kPropagationTest*scale for g in users} 
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.propagation.items()}
        
    def evaluate(self, targetPositions, ageGroup):
        assert isinstance(targetPositions[0], tuple),'tuple list format expected for positions'
        reshapedPos = np.array(self.site).reshape(-1,2)
        # evaluate kernel to get level service score. If age group is not served, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](targetPositions, reshapedPos)
            # check conversion from tuple to nparray
            #targetPositions= np.array(targetPositions)
            #score2 = self.kernel[ageGroup](targetPositions, reshapedPos)
            #assert all(score-score2==0)
        else:
            score = 0
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())

In [ ]:
#outScores = {(a, t): np.zeros(positions.shape[0]) for a in outputAgeGroups for t in outputServices}
#dimSpec = [('position', positions), ('agegroup', outputAgeGroups), ('servicetype', outputServices)]
#outXr = xr.DataArray(np.zeros([positions.shape[0], len(outputAgeGroups), len(outputServices)]),  dimSpec)

### Supply modelling
def evaluate_services_at(positions, unitsList, outputServices= [t for t in ServiceType]):
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()
    # initialise output
    outScores = {service: pd.DataFrame(np.zeros([len(positions), len(AgeGroup.all())]), 
                                 index=positions, columns=AgeGroup.all()) 
                 for service in outputServices}
    # loop over different services
    for thisServType in outputServices:
        for thisAgeGroup in outputAgeGroups:
            serviceUnits = [u for u in unitsList if u.service == thisServType]
            if not serviceUnits:
                continue
            unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)), axis=-1)
            # aggregate unit contributions according to the service type norm
            outScores[thisServType][thisAgeGroup] = thisServType.aggregate_units(unitValues)
            
    return outScores
        

In [ ]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo'), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', position=(45.43, 9.201))]
evaluate_services_at(get_random_pos(50), test)


In [ ]:
class UnitFactory:
    def __init__(self, path):
        assert os.path.isfile(path), 'File "%s" not found' % path
        print("I am your father")
        self.filepath = path
        self.rawData = []
        
    def load(self):
        self.rawData = pd.read_csv(self.filepath, sep=';', decimal=',')
        self.nUnits = self.rawData.shape[0]
        defaultLocationColumns = ['Lat', 'Long']
        if set(defaultLocationColumns).issubset(set(self.rawData.columns)):
            print('Location data found')
            locations = [tuple(self.rawData.loc[i, defaultLocationColumns]) for i in range(self.nUnits)]
            propertData = self.rawData.drop(defaultLocationColumns, axis=1)
        else:
            propertData = self.rawData
            locations = []
            
        return propertData, locations
    
    def log(self, message):
        print(message)
        
    @staticmethod
    def createLoader(serviceType, path):
        if serviceType == ServiceType.School:
            return SchoolFactory(path)


class SchoolFactory(UnitFactory):
    
    def __init__(self, path):
        super().__init__(path)
        
    def load(self):
        (propertData, locations) = super().load()
        print("daughter")
        nameCol = 'DENOMINAZIONESCUOLA'
        unitList = []
        for iUnit in range(self.nUnits):
            # create instance
            thisUnit = ServiceUnit(ServiceType.School, name=propertData[nameCol][iUnit], position=locations[iUnit], 
                        users=AgeGroup.all(), times=[], scale=1)
            unitList.append(thisUnit)
        
        return unitList

In [ ]:
## Load scuole!
oggetto = UnitFactory.createLoader(ServiceType.School, 'data/anagrScuoleMilano_geoloc.csv')
oggetto.load()

In [ ]:
### Demand modelling
class Household:
    def __init__(self, position=None, membersInput=None):
        # make defaults
        if not position: position = get_random_pos(1)
        if not membersInput: membersInput = {a: 1 for a in AgeGroup.all()}
        # expand input to all age group keys
        self.members = {a: membersInput.get(a, 0) for a in AgeGroup.all()}
        self.position = position
        self.export = pd.DataFrame(self.members, index=([self.position])) # precompute for speed

def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand

In [ ]:
hhList =  [Household() for i in range(40)]
evaluate_demand(hhList)

In [ ]:
## Matching demand and supply
def get_satisfaction_indexes(householdList)